# compas_fea2 simple script approach

This example covers the analysis of a simple 2D frame structure generated completely from scratch. 

### NOTE: This example is in SI (m, kg, s)

In [1]:
import compas_fea2

from compas_fea2.model import Model, elements
from compas_fea2.model import Part
from compas_fea2.model import Node
from compas_fea2.model import ElasticIsotropic, Steel
from compas_fea2.model import CircularSection, RectangularSection, MembraneSection
from compas_fea2.model import ShellSection
from compas_fea2.model import BeamElement, MembraneElement, ShellElement
from compas_fea2.model import RollerBCXY
from compas_fea2.model import FixedBC, PinnedBC
from compas_fea2.model import NodesGroup
from compas_fea2.model import BeamEndPinRelease

from compas_fea2.problem import Problem
from compas_fea2.problem import PointLoad
from compas_fea2.problem import GravityLoad
from compas_fea2.problem import FieldOutput
from compas_fea2.problem import StaticStep, ModalAnalysis
from compas_fea2.problem import GeneralDisplacement

from compas_fea2.results import Results
from pathlib import Path
from pprint import pprint

from compas_fea2 import TEMP

## Plug-in registration

compas_fea2 implements a plug-in architecture. The compas_fea2 main package only defines the general API for a Finite Element Analysis, while the actual implementation in the a specific backend is done in the corresponding plug-in, whcih must registered and the beginning of the script. 

In [2]:
compas_fea2.set_backend('abaqus')

Abaqus implementations registered...
Opensees implementations registered...


### Verbosity
you can choose how much info you want to see by setting the `VERBOSE` property
to `True`

In [3]:
compas_fea2.config.VERBOSE = True

## Model

The fist step (of an FEA) is the generation of the analysis `model`. 
A model is always composed by:

1) `Parts`:
2) `Boundary Conditions`:

it can als include:

3) `Constraints` and `Interactions`:
4) `Groups of Parts`:


In [4]:
model = Model()

### Parts

`Parts` are subregions of the model that can be considered to be independent from each other. However, similar parts of a model can still be modelled as different `Parts`. 

A good example of a `Part` could be a brick. 

In [5]:
frame = model.add_part(Part())
print(frame)

AbaqusPart(1922640306912) registered to AbaqusModel(1922640308256).

AbaqusPart
----------
name : AP_1922640306912

number of elements : 0
number of nodes    : 0



Nodes are added to the `Part`. In this example, we want to create a simple  frame with 4 nodes at the base and 1 node at height of 5m.

In [6]:
coordinates = [[0., 0., 5.], [5., -5., 0.], [5., 5., 0.], [-5., 5., 0.], [-5., -5., 0.]]
nodes = [Node(xyz=node) for node in coordinates]

print(nodes[0])

AbaqusNode(1922640064224)


We can now add the material properties we plan to use in our analysis. These material will then be available in the 'Model' and can be used in the definition of the structural sections.

In [7]:
mat = ElasticIsotropic(E=10*10**9, v=0.3, density=1000)

`compas_fea2` uses the same convention as Abaqus where the materials are assigned to the section and not to the elements (sections are assigned to the elements).
The structural properties of the section are computed automatically accordingly to the type of section chosen (`RectangularSection` in this case)

In [8]:
frame_sec = RectangularSection(w=0.05, h=0.1, material=mat)

Structural elements are defined by their type, connectivity and associated section. In this case we use a `BeamElement`, so we need to specify the 2 nodes it is connected to, with the 'BoxSection' defined before.
Elements can be added to the `Part` one by one or as a list. 

In [9]:
beams = [frame.add_element(BeamElement(nodes=[nodes[0], node], section=frame_sec)) for node in nodes[1:]]

Node AbaqusNode(1922640064224) registered to AbaqusPart(1922640306912).
Node AbaqusNode(1922640061104) registered to AbaqusPart(1922640306912).
Element AbaqusBeamElement(1922640184224) registered to AbaqusPart(1922640306912).
SKIPPED: Node AbaqusNode(1922640064224) already in part.
Node AbaqusNode(1922640061056) registered to AbaqusPart(1922640306912).
SKIPPED: Section AbaqusRectangularSection(1922640062688) already in part.
Element AbaqusBeamElement(1922640185472) registered to AbaqusPart(1922640306912).
SKIPPED: Node AbaqusNode(1922640064224) already in part.
Node AbaqusNode(1922640064320) registered to AbaqusPart(1922640306912).
SKIPPED: Section AbaqusRectangularSection(1922640062688) already in part.
Element AbaqusBeamElement(1922640184080) registered to AbaqusPart(1922640306912).
SKIPPED: Node AbaqusNode(1922640064224) already in part.
Node AbaqusNode(1922640062736) registered to AbaqusPart(1922640306912).
SKIPPED: Section AbaqusRectangularSection(1922640062688) already in part.
E

## Initial Boundary Conditions

In [11]:
model.add_pin_bc(nodes=[nodes[1]])
model.add_bcs(bc=FixedBC(), nodes=nodes[2:])

[AbaqusFixedBC(1922659188160),
 AbaqusFixedBC(1922659188160),
 AbaqusFixedBC(1922659188160)]

In [12]:
model.summary()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
compas_fea2 Model: AM_1922640308256
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

description: N/A
author: N/A

Parts
-----
AP_1922640306912
    # of nodes: 5
    # of elements: 4

Interactions
------------
N/A

Constraints
-----------
N/A

Boundary Conditions
-------------------
AP_1922640306912: 
  AbaqusPinnedBC(1922640186864) - # of restrained nodes 1
  AbaqusFixedBC(1922659188160) - # of restrained nodes 3



'\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\ncompas_fea2 Model: AM_1922640308256\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\ndescription: N/A\nauthor: N/A\n\nParts\n-----\nAP_1922640306912\n    # of nodes: 5\n    # of elements: 4\n\nInteractions\n------------\nN/A\n\nConstraints\n-----------\nN/A\n\nBoundary Conditions\n-------------------\nAP_1922640306912: \n  AbaqusPinnedBC(1922640186864) - # of restrained nodes 1\n  AbaqusFixedBC(1922659188160) - # of restrained nodes 3\n'

## Problem definition
The second part of the FEA is to define the `Problem` to be solved. To define a `Problem` you need a `Model` and the `Steps` in which the loads are applied.  

In [13]:

problem = Problem(model=model, name='test')

In [14]:
step_0 = problem.add_step(StaticStep())
step_0.add_gravity_load()

step_1 = problem.add_step(StaticStep())
step_1.add_point_load(x=1000, z=-1000, node=nodes[0])
# # Define the field outputs required
fout = step_1.add_output(FieldOutput())

Once the `Probelm` is set, you can solve it running the analysis. You need to specify the location of the analysis files generated by Abaqus.
The output log for now is limited to the one automatically generated by Abaqus, but the plan is to expand it to have more info about the status of the analysis, possible warnings or errors, etc. 

In [17]:
# # Solve the problem
problem.analyse_and_extract(path=Path(TEMP).joinpath('example'))

SKIPPED: Element AbaqusBeamElement(1922640185472) already in part.
SKIPPED: Element AbaqusBeamElement(1922640184080) already in part.
SKIPPED: Element AbaqusBeamElement(1922640184224) already in part.
SKIPPED: Element AbaqusBeamElement(1922536543664) already in part.
Model generated in  0.0020 secs
Problem generated in  0.0002 secs
Input file generated in: c:\code\myrepos\from_compas\fea2\temp\example\test.inp
Finished writing input file in 0.0062 secs
Analysis initiated from SIMULIA established products
Abaqus JOB test
Abaqus 2021
Begin Analysis Input File Processor
5/31/2022 4:54:52 PM
Run pre.exe
5/31/2022 4:54:58 PM
End Analysis Input File Processor
Begin Abaqus/Standard Analysis
5/31/2022 4:54:58 PM
Run standard.exe
5/31/2022 4:55:03 PM
End Abaqus/Standard Analysis
Begin SIM Wrap-up
5/31/2022 4:55:05 PM
Run SMASimUtility.exe
5/31/2022 4:55:05 PM
End SIM Wrap-up
Abaqus JOB test COMPLETED
Analysis completed in 15.4449 secs

Extracting data from Abaqus .odb file...
Data extracted fro

## Results Visualization

...not there yet... :)

In [23]:
##### --------------------- POSTPROCESS RESULTS -------------------------- #####
problem.show(reactions=0.005)
